In [5]:
import openai, os
import faiss
# from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTFaissIndex, ServiceContext
from llama_index import SimpleDirectoryReader, LangchainEmbedding, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import SpacyTextSplitter, CharacterTextSplitter
from langchain.chat_models import ChatOpenAI

openai.api_key = ""

In [6]:

from llama_index.node_parser import SimpleNodeParser

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model)



Created a chunk of size 130, which is longer than the specified 100


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Use pytorch device: cpu


In [7]:
#dimension = 768
#faiss_index = faiss.IndexFlatIP(dimension)
#index = GPTFaissIndex(nodes=nodes,faiss_index=faiss_index, service_context=service_context)

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import faiss
# dimensions of text-ada-embedding-002
d = 1536 
faiss_index = faiss.IndexFlatL2(d)



In [10]:
from llama_index import SimpleDirectoryReader, load_index_from_storage, GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 3782 tokens


> [build_index_from_nodes] Total embedding token usage: 3782 tokens
> [build_index_from_nodes] Total embedding token usage: 3782 tokens


In [15]:
# save index to disk
index.storage_context.persist()

In [16]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = load_index_from_storage(storage_context=storage_context)

INFO:root:Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.


Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.
Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.
Loading all indices.


ValueError: No index in storage context, check if you specified the right persist_dir.

In [17]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("请问你们海南能发货吗？")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens


> [retrieve] Total embedding token usage: 24 tokens
> [retrieve] Total embedding token usage: 24 tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (2031 > 1024). Running this sequence through the model will result in indexing errors
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2083 tokens


> [get_response] Total LLM token usage: 2083 tokens
> [get_response] Total LLM token usage: 2083 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [18]:
display(Markdown(f"<b>{response}</b>"))

<b>
A: 是的，我们支持全国大部分省份的配送，包括海南。</b>

In [19]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query( "你们用哪些快递公司送货? Y Combinator?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 30 tokens


> [retrieve] Total embedding token usage: 30 tokens
> [retrieve] Total embedding token usage: 30 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2143 tokens


> [get_response] Total LLM token usage: 2143 tokens
> [get_response] Total LLM token usage: 2143 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [20]:
display(Markdown(f"<b>{response}</b>"))

<b>
不，我们与顺丰速运、圆通速递、申通快递、韵达快递、中通快递、百世快递等多家知名快递公司合作，但不包括Y Combinator。</b>

In [3]:



from llama_index import GPTVectorStoreIndex

new_index = GPTVectorStoreIndex(nodes)

# query with embed_model specified
query_engine = new_index.as_query_engine(
    retriever_mode="embedding", 
    verbose=True, 
    service_context=service_context
)



INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 3198 tokens


In [4]:
#from llama_index import QueryMode

#openai.api_key = os.environ.get("OPENAI_API_KEY")

#response = index.query(
#    "请问你们海南能发货吗？", 
#    mode=QueryMode.EMBEDDING,
#    verbose=True, 
#)

response = query_engine.query(
    "请问你们海南能发货吗？", 
#    mode=QueryMode.EMBEDDING,
#    verbose=True, 
)

print(response)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 550 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



是的，我们支持全国大部分省份的配送，其中包括海南省。


In [5]:
response = query_engine.query(
    "你们用哪些快递公司送货？", 
)

print(response)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 27 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 442 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



我们与顺丰速运、圆通速递、申通快递、韵达快递、中通快递、百世快递等多家知名快递公司合作，用他们的服务送货。


In [6]:
#response = index.query(
#    "你们的退货政策是怎么样的？", 
#    mode=QueryMode.EMBEDDING,
#    verbose=True, 
#)

response = query_engine.query(
    "你们的退货政策是怎么样的？", 
)

print(response)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 27 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 438 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。
